<a href="https://colab.research.google.com/github/Yasmine-ChemP/GMCtraining/blob/main/British_broadcast_channel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/susanli2016/PyCon-Canada-2019-NLP-Tutorial/master/bbc-text.csv

--2022-06-11 08:20:38--  https://raw.githubusercontent.com/susanli2016/PyCon-Canada-2019-NLP-Tutorial/master/bbc-text.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5057493 (4.8M) [text/plain]
Saving to: ‘bbc-text.csv’

bbc-text.csv        100%[===================>]   4.82M  --.-KB/s    in 0.07s   

2022-06-11 08:20:38 (67.5 MB/s) - ‘bbc-text.csv’ saved [5057493/5057493]



In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf 
from nltk.corpus import stopwords

import nltk
nltk.download('stopwords')

STOPWORDS= set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df=pd.read_csv('bbc-text.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  2225 non-null   object
 1   text      2225 non-null   object
dtypes: object(2)
memory usage: 34.9+ KB


In [ ]:
print(df["category"].unique())

['tech' 'business' 'sport' 'entertainment' 'politics']


In [ ]:
df.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [ ]:
#recuperation du text 
x=df['text'].values

y= df["category"].values 

In [ ]:
#creation de tensor 
dataset=tf.data.Dataset.from_tensor_slices((x,y))

In [ ]:
for i, r in dataset.take(2):
  print(r.numpy())
  print(i.numpy())


b'tech'
b'tv future in the hands of viewers with home theatre systems  plasma high-definition tvs  and digital video recorders moving into the living room  the way people watch tv will be radically different in five years  time.  that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite pastimes. with the us leading the trend  programmes and other content will be delivered to viewers via home networks  through cable  satellite  telecoms companies  and broadband service providers to front rooms and portable devices.  one of the most talked-about technologies of ces has been digital and personal video recorders (dvr and pvr). these set-top boxes  like the us s tivo and the uk s sky+ system  allow people to record  store  play  pause and forward wind tv programmes when they want.  essentially  the technology allows for much more personalised tv. they are also being built-i

In [ ]:
#labelization of y and vectorize x
from tensorflow.keras.layers import TextVectorization 
#vectorize x 
vectorize_layer=TextVectorization(
    max_tokens=10000,
    #limit the number of words in a review 
    output_sequence_length=200
    

)



In [ ]:
text_ds=dataset.map(lambda x,y:x)
for i in text_ds.take(1):
  print(i.numpy().decode('utf-8'))


tv future in the hands of viewers with home theatre systems  plasma high-definition tvs  and digital video recorders moving into the living room  the way people watch tv will be radically different in five years  time.  that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite pastimes. with the us leading the trend  programmes and other content will be delivered to viewers via home networks  through cable  satellite  telecoms companies  and broadband service providers to front rooms and portable devices.  one of the most talked-about technologies of ces has been digital and personal video recorders (dvr and pvr). these set-top boxes  like the us s tivo and the uk s sky+ system  allow people to record  store  play  pause and forward wind tv programmes when they want.  essentially  the technology allows for much more personalised tv. they are also being built-in to high-

In [ ]:
vectorize_layer.adapt(text_ds)

In [ ]:
len(vectorize_layer.get_vocabulary())

10000

In [ ]:
def vectorize_text(text, label):
  text=tf.expand_dims(text,-1)
  return vectorize_layer(text),tf.reshape(label,(-1,1))

dataset=dataset.map(vectorize_text)

In [ ]:
for text, label in dataset.take(5):
  print(text)
  print(label)

tf.Tensor(
[[ 178  244    7    2 1288    4 1259   18  125 1170  810 5334 1317 4292
     5  220  264 4169 1473   76    2 1392 1718    2  116   46 1003  178
    24   17 6702  406    7  185   80   72   11   10  215    3   35 3186
  1405   36 2571   22    2  658  497 1343  147    7 3002 3122    3 1817
   126  173   48  953   24  827   53    4  124 1013    1   18    2   49
   740    2 1620 1101    5   71  490   24   17 2054    3 1259  841  125
   573  168 1556 2153 1696  212    5  411  197 2783    3  972 5300    5
  1141  682   53    4    2  113    1  953    4 2634   20   39  220    5
   426  264 4169 9605    5 5941  173 4463 3490   97    2   49    8 5868
     5    2   78    8 1537  218  529   46    3  221 1485  153 4687    5
   578 5509  178 1101   65   30  157 6873    2  160 2180    9  139   41
  9251  178   30   26   44   89 4785    3 1317  178 2032   36   26  227
   237    7  493    5    2   49   25 3060    3  114  155    7  224  101
     4    2 1269    4 1317 4484   29   85   55   46  

In [ ]:
label.shape

TensorShape([1, 1])

In [ ]:
#labelize y 
label_vect_layer=TextVectorization( max_tokens=7)

labels= dataset.map(lambda x,y:y)
label_vect_layer.adapt(labels)


def vectorize_label(text, label):
  return text, label_vect_layer(label)[0]
dataset= dataset.map(vectorize_label)
  



In [ ]:

for text, labels in dataset.take(5):
  print(labels)

tf.Tensor([5], shape=(1,), dtype=int64)
tf.Tensor([3], shape=(1,), dtype=int64)
tf.Tensor([2], shape=(1,), dtype=int64)
tf.Tensor([2], shape=(1,), dtype=int64)
tf.Tensor([6], shape=(1,), dtype=int64)


In [ ]:
#creation du model 
inputs=tf.keras.Input(shape=(None,),dtype="int64")

#couche cachée embedding
x=tf.keras.layers.Embedding(10000, 500)(inputs)

#couche LSTM
x=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128))(x)

#couche classification 
x=tf.keras.layers.Dense(64,activation="relu" )(x)

#couche classification 
predictions=tf.keras.layers.Dense(7, activation="sigmoid",name="predictions")(x)

model=tf.keras.Model(inputs,outputs= predictions)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 500)         5000000   
                                                                 
 bidirectional (Bidirectiona  (None, 256)              644096    
 l)                                                              
                                                                 
 dense (Dense)               (None, 64)                16448     
                                                                 
 predictions (Dense)         (None, 7)                 455       
                                                                 
Total params: 5,660,999
Trainable params: 5,660,999
Non-trainable params: 0
___________________________________________________

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=["accuracy"])

In [ ]:
history= model.fit(dataset,epochs=5)

Epoch 1/5
2225/2225 [==============================] - 61s 24ms/step - loss: 0.7440 - accuracy: 0.7272
Epoch 2/5
2225/2225 [==============================] - 54s 24ms/step - loss: 0.1486 - accuracy: 0.9564
Epoch 3/5
2225/2225 [==============================] - 53s 24ms/step - loss: 0.0233 - accuracy: 0.9942
Epoch 4/5
2225/2225 [==============================] - 54s 24ms/step - loss: 0.0052 - accuracy: 0.9973
Epoch 5/5
2225/2225 [==============================] - 53s 24ms/step - loss: 6.9623e-05 - accuracy: 1.0000


In [ ]:
model.evaluate(dataset)

2225/2225 [==============================] - 36s 16ms/step - loss: 1.4495e-05 - accuracy: 1.0000


[1.449480805604253e-05, 1.0]